# Processamento dos arquivos extraídos do Cognos de captação remunerada e de crédito

## Objtetivo

<li> Utilizar os campos de prazo, moeda e taxa para carimbar a codificação do plano funding definitivo
<li> Montar o balanço contábil desdobrado na codificação de contas funding, codificação de moedas e prazos, e PA's

## Import Library

In [1]:
# bibliotecas de manipulação
import pandas as pd
import numpy as np

# bibliotecas de visualização
import matplotlib as plt
import seaborn as sns

# bibliotecas de machine learning

# outras bibliotecas
import datetime
import os
import warnings

In [2]:
# opções para o pandas
pd.options.display.max_columns=None
pd.options.display.max_rows=None
pd.options.mode.use_inf_as_na=True
pd.options.display.max_colwidth=None

# opções para visualizações
%matplotlib inline

# opções para Jupyter notebook
warnings.filterwarnings('ignore')

## Importação de bibliotecas ou classes locais

## Definição de Parâmetros

In [3]:
# define o número do PA que receberá os saldos capturados a partir do balanço contábil
param_pa = 1

# caminho de importação do plano de funding definitivo
path_output_plano='../data/output/plano/plano-funding-definitivo.xlsx'

# caminho de importação do mapeamento entre o plano gerencial da pad e o plano funding
path_input_plano='../data/input/plano/plano-funding-pad.xlsx'

# caminho de importação dos arquivos do cognos de captação remunerada e crédito
path_input_files='../data/input/funding/{}'

# caminho de gravação dos arquivos de funding processados
path_output_files='../data/output/balanco/{}'

# caminho de importação dos arquivos contábeis
path_input_contabil='../data/input/contabil/{}'

# recebe as críticas do processamento
df_erros=pd.DataFrame(columns=['DataHora', 'Mensagem', 'Impeditivo'])

## Funções

In [4]:
# devolve um array com o código do prazo, taxa e moeda do funding pesquisado
def obter_codmoeda(dic_valores):
    codMoeda=df_plano_funding[
        (df_plano_funding.CtaFunding==dic_valores['CtaFunding']) &
        (df_plano_funding.Origem==dic_valores['Origem']) &
        (df_plano_funding.TipoMoeda==dic_valores['Moeda']) &
        (df_plano_funding.VlrPrazoInicial <= dic_valores['DiasVencimento']) &
        (dic_valores['DiasVencimento'] <= df_plano_funding.VlrPrazoFinal)
    ].CodMoeda.values
    
    if len(codMoeda) > 0:
        codMoeda=codMoeda[0]
    else:
        codMoeda=None
    
    return codMoeda

def obter_codfunding(dic_produto):
    codFunding=df_depara[
        (df_depara.Tipo==dic_produto['Tipo']) &
        (df_depara.Modalidade==dic_produto['Modalidade']) &
        (df_depara.Submodalidade==dic_produto['Submodalidade'])
    ].CodFunding.values
    
    if len(codFunding)>0:
        codFunding=codFunding[0]
    else:
        codFunding=None
    return codFunding

# adiciona uma mensagem de validações de conferências e críticas ao dataframe de erros
def add_msg(mns, impedimento=0):
    return pd.DataFrame(
        [datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S"), mns, impedimento],
        index=['DataHora', 'Mensagem', 'Impeditivo']
        ).T

## Importação de Dados e processamento de dados originados do produto

In [5]:
# obtendo os nomes dos arquivos a serem importados
list_files=os.listdir(path_input_files.format(''))


# importando plano funding
df_plano_funding=pd.read_excel(path_output_plano, index_col=0)


# importação tabela mapeamento produtos cognos para plano funding
df_depara=pd.read_excel(path_input_plano, sheet_name='De-Para')


# dataframe que receberá os arquivos de importação de origens e aplicações
df_funding=pd.DataFrame()


# importando arquivos extraídos do cognos de origens a aplicações
for file in list_files:
    df_funding=df_funding.append(
        pd.read_csv(
            path_input_files.format(file),
            header=0,
            sep='\t',
            encoding='utf-16-le',
            decimal=','
        )
    )
df_funding.DtMovimento=pd.to_datetime(df_funding.DtMovimento, format="%Y/%m/%d")
df_funding.reset_index(inplace=True, drop=True)


# usando os códigos dos produtos para obter os códigos do plano funding
df_funding['CtaFunding']=df_funding.apply(lambda x:
                                          obter_codfunding(x[['Tipo','Modalidade', 'Submodalidade']].to_dict()), axis=1)

# carimbando a codificação de prazo, taxa e moeda do recurso/aplicação localizada no plano funding
df_funding['CodMoeda']=df_funding.apply(
    lambda x: obter_codmoeda(x[['DiasVencimento','CtaFunding', 'Origem', 'Moeda']].to_dict()), axis=1
)


# agregando valores
df_funding=df_funding.groupby(by=['NumCooperativa', 'NumPA','DtMovimento', 'CtaFunding','CodMoeda'])[ 'SMDC', 'SF', 'VlrJuros'].sum().reset_index()


# fazendo merge com plano funding
df_funding=df_funding.merge(
    df_plano_funding[
        [
            'CodMoeda',
            'CtaFunding',
            'Nome',
            'CtaDesdobramento',
            'BolDesdobrarConta',
            'Fonte',
            'TipoSaldo',
            'Origem',
            'TipoMoeda',
            'VlrPrazoFinal',
            'NomeMoeda',
            'OrdemUtilizacaoFunding',
            'NumOrdemUtilizacaoMoeda'
        ]
    ],
    how='left',
    left_on=['CodMoeda', 'CtaFunding'],
    right_on=['CodMoeda', 'CtaFunding']
)

# ajustando posição de colunas
df_funding=df_funding[[i for i in df_funding.columns if i not in ['SMDC','SF','VlrJuros']] + ['SMDC','SF','VlrJuros']]

# ajustando datatype
df_funding.CtaDesdobramento.fillna(-1, inplace=True)
df_funding.CtaDesdobramento=df_funding.CtaDesdobramento.astype('int')


# correção da natureza do SMDC conforme atributos do plano funding
df_funding.SMDC=df_funding.apply(
    lambda x:
    -x.SMDC if x.TipoSaldo=='D' else x.SMDC, axis=1
)


# correção da natureza do SF conforme atributos do plano funding
df_funding.SF=df_funding.apply(
    lambda x:
    -x.SF if x.TipoSaldo=='D' else x.SF, axis=1
)


# correção da natureza do VlrJuros conforme atributos do plano funding
df_funding.VlrJuros=df_funding.apply(
    lambda x:
    -x.VlrJuros if x.TipoSaldo=='C' else x.VlrJuros, axis=1
)

## Importação de Dados Contábeis da PAD

In [6]:
# obtendo os nomes dos arquivos a serem importados
list_files=os.listdir(path_input_contabil.format(''))


# importando plano funding
df_plano_gerencial=pd.read_excel(path_input_plano, sheet_name='PlanoGerencial')
df_plano_gerencial.drop(columns=['NomeCtaGerencial'], inplace=True)


# dataframe que receberá os arquivos de contábeis da PAD
df_contabil=pd.DataFrame()


# importando arquivos extraídos do cognos de origens a aplicações
for file in list_files:
    df_contabil=df_contabil.append(
        pd.read_csv(
            path_input_contabil.format(file),
            header=0,
            sep='\t',
            encoding='utf-16-le',
            decimal=','
        )
)
df_contabil.DtMovimento=pd.to_datetime(df_contabil.DtMovimento, format="%Y/%m/%d")
df_contabil.reset_index(inplace=True, drop=True)
df_contabil.drop_duplicates(inplace=True)


# criando unindo plano gerencial aos dados da PAD
df=df_contabil.merge(
    df_plano_gerencial,
    how='inner',
    left_on=['NumCtaGerencial'],
    right_on=['NumCtaGerencial']

)
df.sort_values(by=['NumCooperativa', 'NumCtaGerencial'], inplace=True)
df.reset_index(inplace=True, drop=True)


# Ajustando saldo de contas patrimoniais em função da natureza
metricas_saldo=['SMDU', 'SMDC', 'SF']
for metrica in metricas_saldo:
    df[metrica]=df.apply(
        lambda x: -abs(x[metrica]) if x.BolSaldoDevedor==1 and x.Natureza in (['P', 'A']) else abs(x[metrica]), axis=1
    
    )

    
# Capturando o valor dos juros do balanço contábil para as respectivas rubricas patrimoniais
df['VlrJuros']=df.apply(
    lambda x:
    df[
        (df.NumCooperativa==x.NumCooperativa) &
        (df.DtMovimento==x.DtMovimento) &
        (df.NumCtaGerencial==x.CtaJuros)
    ].VlrMovimento.sum(), axis=1
)


# eliminando contas de resultado do dataframe
df=df[df.Natureza.isin(['P','A'])]


# agregando balanço nas contas funding
df=df.groupby(
    by=['NumCooperativa', 'DtMovimento', 'CtaFunding']
)[['SMDU', 'SMDC', 'SF', 'VlrJuros']].sum().reset_index().sort_values(by=['NumCooperativa','DtMovimento', 'CtaFunding'])
df.CtaFunding=df.CtaFunding.astype('int')
    

    
# processar crítica de mapeamento das contas da pad pra checar se falta mapear alguma conta nova
# agregando mapeamento das contas da PAD pra identificar se há divergência entre ativos e passivos
i=df.groupby(by=['NumCooperativa', 'DtMovimento'])[['SMDU', 'SMDC', 'SF', 'VlrJuros']].sum().round(2).reset_index()

colunas=['SMDU', 'SMDC', 'SF']
for row in range(i.shape[0]):
    for col in colunas:
            
            # gravando validação de saldos
            msg=str(i.iloc[row].NumCooperativa) + ' ' + str(i.iloc[row].DtMovimento) + ' ' + ' validando divergência entre ativos e passivos no mapeamento de contas gerenciais da PAD {}: {}'.format(col, str(i.iloc[row][col]))
            
            # checando se  diferença no saldo são maiores que 100, se sim, impedir processamento
            impedimento=1 if abs(i.iloc[row][col]) > 100 else 0
            df_erros=df_erros.append(
                add_msg(msg, impedimento)
            )


# adicionando os dados de balanço em dataframe temporário a serem levados para o dataframe de funding em próxima etapa
df_balanco=df.merge(
    # fazendo merge com plano de funding para capturar as codificações de prazos, moedas e taxas a serem levadas para o dataframe de funding
    df_plano_funding[(df_plano_funding.Fonte=='B') & (df_plano_funding.TipoSaldo!='DC')][[
        'CtaFunding', 'CodMoeda', 'Nome', 'CtaDesdobramento', 'BolDesdobrarConta','Fonte','TipoSaldo', 'Origem', 'TipoMoeda', 'VlrPrazoFinal', 'NomeMoeda', 'OrdemUtilizacaoFunding',
        'NumOrdemUtilizacaoMoeda'
    ]],
    how='left',
    left_on=['CtaFunding'],
    right_on=['CtaFunding']
    
)
df_balanco.drop(columns='SMDU', inplace=True)


# atribuindo número do PA = UAD para as contas provenientes de balanço. Os saldos de produto permanecem nos PA's
df_balanco['NumPA'] = param_pa

# reorganizando posição de colunas
df_balanco=df_balanco[[
    'NumCooperativa', 'NumPA', 'DtMovimento','CtaFunding','CodMoeda','Nome','CtaDesdobramento','BolDesdobrarConta','Fonte','TipoSaldo','Origem','TipoMoeda','VlrPrazoFinal',
    'NomeMoeda','OrdemUtilizacaoFunding','NumOrdemUtilizacaoMoeda','SMDC','SF','VlrJuros'
]]


# selecionando as contas que não serão desdobradas a partir de fontes do cognos
df_balanco=df_balanco[df_balanco.BolDesdobrarConta==0]

## Pre Processamento

In [7]:
# agregando saldos originados do cognos nas contas agregadoras para comparar com a fonte contábil
df_tmp=df_funding.groupby(
    by=['NumCooperativa', 'DtMovimento', 'CtaDesdobramento']
)[['SMDC', 'SF', 'VlrJuros']].sum().reset_index().rename(
    columns={'SMDC': 'SMDC_Produto', 'SF': 'SF_Produto','VlrJuros': 'VlrJuros_Produto', 'CtaDesdobramento': 'CtaFunding'}
)


# unindo somatórios provenientes do produto ao dataframe contábil para fazer a conciliação
df_processamento=df.merge(
    df_tmp,
    how='left',
    left_on=['NumCooperativa', 'DtMovimento', 'CtaFunding'],
    right_on=['NumCooperativa', 'DtMovimento', 'CtaFunding']
)


# calculando divergência de SMDC
df_processamento['SMDC_Diff']=(df_processamento.SMDC - df_processamento.SMDC_Produto).round(2)

# calculando divergência de SF
df_processamento['SF_Diff']=(df_processamento.SF - df_processamento.SF_Produto).round(2)

# calculando divergência de juros
df_processamento['VlrJuros_Diff']=(df_processamento.VlrJuros - df_processamento.VlrJuros_Produto).round(2)

# define o PA padrão para o qual os lançamentos de ajuste de saldo entre produtos e contábil será lançado
df_processamento['NumPA'] = param_pa


# atribui a dataframe temporário contas de funding que serão lançadas as divergencias para ajuste e conciliação de saldo
df_tmp=df_plano_funding[
    (df_plano_funding.TipoSaldo=='DC') & (df_plano_funding.Fonte=='B')
][['CtaFunding', 'CodMoeda', 'Nome', 'CtaDesdobramento','BolDesdobrarConta','Fonte','TipoSaldo', 'Origem', 'TipoMoeda', 'VlrPrazoFinal', 'NomeMoeda', 'OrdemUtilizacaoFunding',
  'NumOrdemUtilizacaoMoeda']]
df_tmp.CtaDesdobramento=df_tmp.CtaDesdobramento.astype('int')


# cria dataframe de ajuste com base nas contas funding do dataframe temporario para preparar os lançamentos de ajuste para df_funding
df_ajuste=df_processamento.merge(
    df_tmp,
    how='left',
    left_on=['CtaFunding'],
    right_on=['CtaDesdobramento']   
)[['NumCooperativa', 'NumPA', 'DtMovimento', 'CtaFunding_y', 'CodMoeda','Nome', 'CtaDesdobramento', 'BolDesdobrarConta','Fonte','TipoSaldo','Origem', 'TipoMoeda', 'VlrPrazoFinal',
   'NomeMoeda','OrdemUtilizacaoFunding', 'NumOrdemUtilizacaoMoeda','SMDC_Diff', 'SF_Diff', 'VlrJuros_Diff'
]].rename(columns={'CtaFunding_y':'CtaFunding', 'SMDC_Diff': 'SMDC', 'SF_Diff': 'SF', 'VlrJuros_Diff':'VlrJuros'})


# filtra apenas as contas de ajuste entre saldos contábeis versus saldos de produto
df_ajuste=df_ajuste[df_ajuste.TipoSaldo.isin(['DC'])]
df_ajuste.CtaFunding=df_ajuste.CtaFunding.astype('int')
df_ajuste.CtaDesdobramento=df_ajuste.CtaDesdobramento.astype('int')

## Acrescentando ajustes e dados de balanço ao dataframe de funding

In [8]:
# acrescentando os ajustes ao funding
df_funding=df_funding.append(df_ajuste)


# acrescentando os dados provenientes do balanço
df_funding=df_funding.append(df_balanco)

df_funding.sort_values(by=['NumCooperativa', 'DtMovimento', 'CtaFunding', 'NumOrdemUtilizacaoMoeda'], inplace=True)
df_funding.reset_index(inplace=True, drop=True)

# calculando valor dos juros de cada fundo
df_funding['VlrTaxaJuros']=df_funding.apply(lambda x: x.VlrJuros / x.SMDC if x.SMDC!=0 else 0, axis=1)

# convertendo código de moeda para inteiro para join em etapa subsequente
df_funding.CodMoeda=df_funding.CodMoeda.astype('int64')

## Validando erros no dataframe funding final

In [9]:
# filtrando registros com alguma coluna em branco, exceto CtaDesdobramento
i=df_funding[pd.isnull(df_funding.drop(columns='CtaDesdobramento')).any(axis=1)].groupby(by=['NumCooperativa', 'DtMovimento']).count().reset_index()[['NumCooperativa', 'DtMovimento']]

for row in range(i.shape[0]):
    msg=str(i.iloc[row].NumCooperativa) + ' ' + str(i.iloc[row].DtMovimento) + ' Colunas nulas localizadas no df_funding.'
    impedimento=1
    df_erros=df_erros.append(add_msg(msg, impedimento))

    
# validando divergências nos saldos médios e finais do dataframe final de funding
i=df_funding.groupby(by=['NumCooperativa', 'DtMovimento'])[['SMDC', 'SF', 'VlrJuros']].sum().round(2).reset_index()
colunas=['SMDC', 'SF']
for row in range(i.shape[0]):
    for col in colunas:
            
            # gravando validação de saldos
            msg=str(i.iloc[row].NumCooperativa) + ' ' + str(i.iloc[row].DtMovimento) + ' ' + ' validando divergência entre ativos e passivos no dataframe final de funding {}: {}'.format(col, str(i.iloc[row][col]))
            
            # checando se  diferença no saldo são maiores que 100, se sim, impedir processamento
            impedimento=1 if abs(i.iloc[row][col]) > 100 else 0
            df_erros=df_erros.append(
                add_msg(msg, impedimento)
            )

## Output

In [10]:
check=df_erros.Impeditivo.max()

if check==0:
    df_funding.to_excel(path_output_files.format('balanco-funding.xlsx'), sheet_name='funding', freeze_panes=(1,1))
    
    # criando arquivo log de conclusão com sucesso
    with open(path_output_files.format('log.txt'), 'w', encoding='utf-8') as f:
        f.write('Balanço funding gerado com sucesso.')
        f.close()
else:
    
    # criando arquivo log de conclusão com ressalva
    with open(path_output_files.format('log.txt'), 'w', encoding='utf-8') as f:
        f.write('Foram encontrados erros impeditivos no balanço final do funding. Cheque o dataframe de erros.')
        f.close()
        
df_erros

,DataHora,Mensagem,Impeditivo
0,"10/22/2021, 09:03:25",1234 2021-08-31 00:00:00 validando divergência entre ativos e passivos no mapeamento de contas gerenciais da PAD SMDU: -0.0,0
0,"10/22/2021, 09:03:25",1234 2021-08-31 00:00:00 validando divergência entre ativos e passivos no mapeamento de contas gerenciais da PAD SMDC: 0.0,0
0,"10/22/2021, 09:03:25",1234 2021-08-31 00:00:00 validando divergência entre ativos e passivos no mapeamento de contas gerenciais da PAD SF: 0.0,0
0,"10/22/2021, 09:03:25",1234 2021-08-31 00:00:00 validando divergência entre ativos e passivos no dataframe final de funding SMDC: 0.01,0
0,"10/22/2021, 09:03:25",1234 2021-08-31 00:00:00 validando divergência entre ativos e passivos no dataframe final de funding SF: 0.0,0
